
> **This notebook will detail the complete process for exactly how a single station-event is downloaded, corrected by ATaCR, corrected by HPS, and compared with eachother. It can serve as a precise reference (a cookbook) on the specific actions all executed code takes at any stage in the download and corrections to a single station-event and the subsequent comparison of those results.**

## Data
For this example station-event correction and method comparison, 

The station will be:
* Station:	M07A
* Network:	7D
* Latitude:	44.898701
* Longitude:	-125.116798
* Experiment:	CASCADIA INITIATIVE
* Instrument_Design:	AB
* Seismometer:	Trillium Compact
* Environment:	North Pacific
* Pressure_Gauge:	DPG
* Water_Depth_m:	1356.5

The event will be:
* Event:	2012.181.21.07
* M:	6.3
* depth:	17.1
* ev_lla:	43.4335
* ev_lon:	84.6125

> Nothing in this correction and comparison process is designed to any specific station or event properties.

## Procedure Overview

This entire process is done, in sequence, in four phases:

> Phase One: Download data
> 
> Phase Two: ATaCR Correction
> 
> Phase Three: HPS Correction
> 
> Phase Four: Compare Results 

## Procedure

### `Phase One: Download data`

        Step 1.1: Days randomly selected for ATaCR noise download

> Using the station's deployment times recorded at IRIS DMC, an external administrator function called [ObsQA.TOOLS.io.DownloadDayNoise](https://github.com/charleshoots/ATACR_HPS_Comp/blob/512a17e70cc906809bddecbc30767b5a219f0a8a/Packages/CompCode/ObsQA/TOOLS/io.py#L858) randomly selects 15 days then calls ATaCR's [atacr_download_data](https://github.com/charleshoots/ATACR_HPS_Comp/blob/512a17e70cc906809bddecbc30767b5a219f0a8a/Packages/ATaCR/OBStools/obstools/scripts/atacr_download_data.py) script to download these days. To ensure the requested noise population is always satisfied and protected from errors caused by data gaps, [atacr_download_data](https://github.com/charleshoots/ATACR_HPS_Comp/blob/512a17e70cc906809bddecbc30767b5a219f0a8a/Packages/ATaCR/OBStools/obstools/scripts/atacr_download_data.py) is called recursively until atleast 15 days of noise from each channel (Z,P,1,2) is downloaded and saved to SAC. If any day fails to download, a new one is randomly selected.

>> [Note] 10 days is the minimum target used in the noise spectra calculations. Thus, we download 15 days of noise in case some are removed in the QC steps.
 
>> [Note] the random selection is made with a pre-defined seed so the same selections can be repeated for a given station.

        Step 1.2: Noise downloaded, pre-processed, then saved to SAC.

> Within [atacr_download_data](https://github.com/charleshoots/ATACR_HPS_Comp/blob/512a17e70cc906809bddecbc30767b5a219f0a8a/Packages/ATaCR/OBStools/obstools/scripts/atacr_download_data.py), a standard download procedure is implimented for 24-hr four-component seismic/pressure data.

> Data is downloaded and processed in this exact sequence:

1. 24-hr seismic (Z,1,2) traces are downloaded by the IRIS client.
2. In a seperate request to IRIS, 24-hr pressure data is downloaded by the client.
3. All channels detrended (linear & demean).
4. A 2-corner zero-phase anti-alias filter is applied to each channel at 2.5hz.
5. Each channel is resampled to 5hz.
6. Timing is checked on all channels to verify perfect overlap. The code is vulnerable to data gaps without this QC step.
7. Lastly, instrument response is removed from all four channels then saved to seperate SAC files.


        Step 1.3: 2-hr Event (used by ATaCR) downloaded, pre-processed, then saved to SAC.

> Following the exact same set of pre-processing steps but for 2-hr event traces, [atacr_download_event](https://github.com/charleshoots/ATACR_HPS_Comp/blob/512a17e70cc906809bddecbc30767b5a219f0a8a/Packages/ATaCR/OBStools/obstools/scripts/atacr_download_event.py) is called for the chosen event.

        Step 1.4: 24-hr Event (used by HPS) downloaded, pre-processed, then saved to SAC.

> To ensure the exact same pre-processing steps used in ATaCR are always applied to the 24-hr event traces used by HPS, the same script used by ATaCR to download 24-hr noise in step 1.2 ([atacr_download_data](https://github.com/charleshoots/ATACR_HPS_Comp/blob/512a17e70cc906809bddecbc30767b5a219f0a8a/Packages/ATaCR/OBStools/obstools/scripts/atacr_download_data.py)) is directed to a 24-hr window that covers the complete day the chosen event occured. The SAC outputs are saved to a separate folder.

>> [Note] a reasonable alternative would have been to have ATaCR download its events in 24-hr traces by default and trim them at some later step. However, this would have required changing parts of the download, daily-spectra, and event correction scripts and significantly slowing down ATaCR overall. My approach is simpler and gaurantees identical pre-processing between ATaCR/HPS while still being non-invasive to the ATaCR mechanics and run speed.

Stages:
1. Download data
2. ATaCR
    1. Download noise
    2. Download event
    3. Quality-control of noise spectra
    4. Average spectra
    5. Calculate tilt corrective transfer functions
    6. Calculate compliance corrective transfer functions
    7. Correct event
3. HPS
4. Compare results

### `Phase Two: ATaCR Correction`

Phase Two is implimented in three steps:
    2.1: atacr_daily_spectra | Calculation of noise spectra.
    2.2: atacr_clean_spectra | Quality control of noise spectra.
    2.3: atacr_transfer_functions | Calculate tilt corrective transfer functions

        Step 2.1: atacr_daily_spectra | Calculate noise spectra

> Using [atacr_daily_spectra](https://github.com/charleshoots/ATACR_HPS_Comp/blob/512a17e70cc906809bddecbc30767b5a219f0a8a/Packages/ATaCR/OBStools/obstools/scripts/atacr_daily_spectra.py), this step yields the daily spectra for each of the 10 days of downloaded noise.

> For each day, stft's are made of each of the four channels in 2-hr windows with a 30% overlap. Using the log-PSDs for each, windows with an std-norm greater than 1.5 times the day's std are labeled for removal in the next step (2.2). If less than 10 good windows exist, the day is not used. 

> A QC figure of all fft segments (good and bad) is saved to show this.

# Figure: QC of 2-hr STFT segments (7D.M07A.2012.164.QC)
![Figure](znb_images/7D.M07A.2012.164.QC.png)

> Using the good windows, the day's average of power and cross channel spectra are calculated and stored.

> The day's average of power and cross channel spectra for Z and P are then calculated with respect to the horizontal channels rotated parallel to the direction of maximum coherence with Z. 

> Lastly, the day's average power spectra (with bad windows removed) is calculated and saved to a figure output.

>> [Note] The day's average coherence/phase/admittance are also plotted but are not shown here for the sake of brevity.

>> [Note] these ZH and PH spectra and the subsequent related transfer functions are only used in the day-average event corrections. This example will use station averaged transfer functions, so these are never used. Nonetheless, the ATaCR code still calculates and stores everything necessary for either (day or station average) event correction approach.

# Figure: A single day's average power spectra (7D.M07A.2012.164.average)
![Figure](znb_images/7D.M07A.2012.164.average.png)

        2.2: atacr_clean_spectra | Quality control of noise spectra.

> Using [atacr_clean_spectra](https://github.com/charleshoots/ATACR_HPS_Comp/blob/512a17e70cc906809bddecbc30767b5a219f0a8a/Packages/ATaCR/OBStools/obstools/scripts/atacr_clean_spectra.py), the station's average noise spectra are modeled.

>First, all 15 days of day-averaged noise for this station are loaded.

>Similar to the QC stage in 2.1, days with component PSDs that have an std>1.5x the station average are labeled for removal. Additionally, an f-test is made againstall day-noise variances to test for an equal distribution of power on the PSDs. Days with less than a 95% confidence interval are also labeled for removal.

>With the good days defined, the station-averaged power and cross spectra are calculated and stored. As mentioned with 2.1, HH/HZ/HP are also calculated though never used with station-average corrections.

>Lastly, figures are made for station's average admittance, phase, and coherence.

# Figure: QC of station daily power specta used to make the station average 
# Red are bad days.
![Figure](znb_images/7D.M07A.QC.png)

# Figure: Station Averaged Coherence
![Figure](znb_images/7D.M07A.av_coherence.png)

# Figure: Station Averaged Admittance
![Figure](znb_images/7D.M07A.av_admittance.png)

# Figure: Station Averaged Phase
![Figure](znb_images/7D.M07A.av_phase.png)

## `[Troubleshooting Addendum]: Instrument Response`

### TROUBLESHOOTING OBJECTIVES: 
> ONE: Describe the issue

> TWO: Through examination, determine the exact location of the issue how it presents (e.g. exactly what channels and what metrics show it)

> THREE: Find a testable cause and solution to the issue.

> FOUR: Prove efficacy of the solution through testing.
___
## OBJECTIVE ONE: Describe the issue
___
> Based on three observations something may be at issue between the noise/event download (1.2) and average spectra (2.2) stages in how we use Py-ATaCR. 
> 1. The amplitudes in noise spectra for pressure, 
> 2. the behavior of any phase calculated wrt pressure, 
> 3. the amplitudes for any admittance wrt to pressure.


`Comment`: An initial hypothesis is that the most likely culprit is the remove_response correction by obspy and ONLY when used on the pressure data.

## Re-plot of the M07A average spectra figure in 2.2 (above) for quicker reference: 
![Figure](znb_images/7D.M07A.QC.png)

___
## OBJECTIVE TWO: Through examination, determine the exact location of the issue and how it presents (e.g. exactly what channels and what metrics show it)
___
First, let's establish whether or not the issue is not with seismic and thus a result how of pressure is handled between stages 1.2 and 2.2.

>Below is a plot clipped directly from MUSTANG for Z, 1, and 2 channels over M07A's entire deployment, showing each channel's log-power PSD PDF usually sitting between -80 and -160 dB. 
>The lack of a 10x scale missing when ATaCR plotted the log-power spectra (above) and just generally being difficult to read not withstanding, the horizontal channels appear consistent with the spectra shown here in MUSTANG.

>> Comparing the vertical channel to this mustang output, I think the microseism peaks in each plot (-10 in ATaCR and -100 dB in mustang at .1hz / 10s) are very consistent with eachother. 

>> The infravity peak (about -10 at 100s) on the vertical around 100s shown in the ATaCR average spectra plot looks quite high relative to what we see in mustang (-120 to -150 dB in the same range). The plot ticks are just not great for reading in this figure from ATaCR... so maybe they are consistent? 

>> Regardless, the values are still within a reasonable range and thus out of the scope of what I am looking for right now.

>> Based on the seismic spectra alone, I will move forward assuming that atleast the seismic is showing up as we would expect it to.

![Figure](znb_images/7D.M07A.Mustang.png)

Most important summary observation is that each have <0 dB (decimal density values <1) spectra for the notch and power >1hz. This is not the case for M07A output spectra plot from ATaCR.

`[Observation]`: Furthermore when looking at spectra, a good indicator that the spectra for differential pressure gauges are calculated correctly might be small changes where noise is lowest, specifically the noise notch. This would present as <0 dB amplitudes on a log-scale.

This combined with the examination of the seismic (above), builds the argument that the issue is contained to only the pressure data. 

Pressure is a bit more complicated as MUSTANG (and OBSIC) do not have spectra for the pressure channel on this particular station. That said, just the range of reasonable values for pressure spectra at a similar station may be enough for my needs.

Here is the PSD PDFs from OBSIC for three different station differential-pressure gauges (all from ACCSE) of similar depths to M07A (1356m).

OBSIC Pages:

WD48 (1324m): https://obsic-metrics.whoi.edu/XO/WD48

WD55 (1283m): https://obsic-metrics.whoi.edu/XO/WD55

WS75 (1108m): https://obsic-metrics.whoi.edu/XO/WS75

> Summarizing their behaviors overall, peaks around 20-40 dB at periods about >10s and again for the microseism from 1-10s at about 25 dB. More importantly, each have <0 dB (decimal density values <1) spectra for the notch and power >1hz. 

>> This is not at all the case for M07A output spectra plot from ATaCR shown above.

![Figure](znb_images/OBSIC.Reference.Stations.Pressure.png)


> Between the ATaCR download steps (1.2) and station average spectra (2.2), with the exception of the remove_response function in the event/noise download scripts, I've confirmed Py-ATaCR treats the seismic and pressure channels simultaneously by the exact same functions with zero distinctions to the different channels. 

> This was confirmed by looking at the day-spectra script which is just a single for loop run over all four channels.

> From this, it's atleast clear that the issue is not only with the pressure data but specifically the remove_response function when it operates on the pressure (not seismic) data that exist in both download scripts.

#### `Summary conclusion to objective two: The issue is ONLY on the pressure channel.`

___
## OBJECTIVE THREE: Find a possible cause and solution
___

With that established, we can now focus on the remove_response function when applied to pressure. 

We find these are the only two specific lines in the Python ATaCR repo (on Pascal Audet's page, links below) that call remove_response on the pressure channel data:

(in the noise download script) https://github.com/nfsi-canada/OBStools/blob/7a08e26887f576c6d9042cdb01b922844423107a/obstools/scripts/atacr_download_data.py#L611

(in the event download script) https://github.com/nfsi-canada/OBStools/blob/7a08e26887f576c6d9042cdb01b922844423107a/obstools/scripts/atacr_download_event.py#L670

It is very important to acknowledge that neither of these calls to remove_response define the desired units of output. This is an issue because, by default, the function assumes you want output units in VELOCITY unless told otherwise.

remove_response function doc: https://docs.obspy.org/packages/autogen/obspy.core.trace.Trace.remove_response.html

This as a start to a working theory for the cause of the issue shown with pressure, you may also note on the code docs page that remove_response also uses a water-level approach in the response removal, which essentially clips the spectra above a specified dB value relative to the maximum in that bin. 

The consequences of this (water-level) when applied to data in the way that we have been is discussed in depth by the obspy-devs here: https://github.com/obspy/obspy/issues/3136

___

### In summary of this possible cause/solution, following the recommendations on the docs page:

> #### It is possible (likely) ATaCR should of always had the pressure remove_response water-level disabled (set to None) and/or set units to 'DEF' (input/output units are left the same).

> #### We will now test the efficacy of this solution below. If it works, we expect to see a pressure spectra that maps consistently with ranges shown in the above reference stations.

>> [Note]: At some point in the future, the remove_response needs to be removed from the download steps of Py-ATaCR and placed in the day-spectra step. This would make QA on these intermediate pre-processing steps, that ultimately lead to this problem, more practical. Everything has to be re-downloaded because of this. That said, to minimize adding more tweaks than necessary just for this test, I won't do this for now.

___
## OBJECTIVE FOUR: Test the solution
___
Here, we see the ATaCR output for station average spectra (the QC plot) when re-running ATaCR (and clearing data archive in each run) for three different setups in the call to remove_response for the pressure channel.

`[Note]` Each test below each use identical days as the original with a few more added as the bar for noise download after the original dataset was built was raised from 10 to 12.

### Test designs
`[Test-1]`:  Made by disabling the water-level approach in the instrument response removal aswell as forcing it to never add zeros to the poles (by setting output units equal to input units with output='DEF'). 

`[Test-2] and [Test-3]`: Made by running the two additional arguments (water_level=None, output='DEF') seperately in independent ATaCR runs. 

### Test results for station average spectra (below)
`[Test-1]` This setup produces spectra well within the range of values we see in the reference stations above.

`[Test-2] and [Test-3]`:This results in only minor dB changes that are just as inconsistent with the reference stations as the original is.
___
> ![Figure](znb_images/AVG.Spectra.Comparisons.png)



___

### Results on coherence, admittance, and phase: 

> Coherence: Appears less sensitive than the other two metrics to this change in instrument response, with the most obvious changes shown in only a few days with different coherence values around 10s period.

> Admittance: Several magnitudes in changes are shown in admittance with any that were done against the pressure channel spectra (1P, 2P, and ZP). 

> Phase:  the phase is fundamentally different (fixed?) now in Test-1, lacking the double phase trends.

### Broken down by tests:

> **[Test-1]:** Just as with the average spectra tests above, Test-1 shows the greatest change in coherence, admittance, and phase.

> **[Tests -2 and -3]:** Likewise, -2 and -3 are nearly identical output to eachother and show the least change in the three metrics. 
>> Though these two tests likely show the least agreement with the reference stations as pointed out above, these two tests also resulted in the removal of the double phase trend shown in the original ATaCR setup.

`[Comment]`: It might help if I had the coherence/admittance outputs from the Matlab ATaCR to compare these tests. I will run ML-ATaCR if these tests still call for it.
 


### Overall findings of solution testing: 

> Only with Test-1 do we see spectra that is highly consistent with the reference stations. 

> Across all three tests we also see the irregular phase double trend removed.

> Across all three tests we see the amount of days removed by the QC step (plotted in red) is now zero

> In total, these tests seem to indicate that the ATaCR code needs to be changed to be consistent with the execution done in Test-1.
___

> ![Figure](znb_images/Original.Coh.Adm.Ph.png)
___
> ![Figure](znb_images/Test1.Coh.Adm.Ph.png)
___
> ![Figure](znb_images/Test2.Coh.Adm.Ph.png)
___
> ![Figure](znb_images/Test3.Coh.Adm.Ph.png)

___
# Conclusions:

`|1|` Both Python ATaCR download scripts contain an error in how the instrument response is removed from the pressure channel. 

`|2|` It is fixed by changing the two calls for it in the noise/download scripts to the following: 

`stp.remove_response(pre_filt=args.pre_filt,output='DEF',water_level=None)`

`|3|` This change to the code produces pressure channel spectra now highly consistent with spectra from both OBSIC on reference stations of similar depths and published results (e.g. Fig. 4 in Janiszewski et al, 2019) aswell as fixes the irregular behaviors originally shown in the admittance and phase plots.

Additional Notes:

> 1. At some point, the instrument response should be moved from the download scripts to the start of the day-spectra script. It would help alot with QC/QA on these intermediate processing steps of ATaCR.

> 2. It's out of the scope of my own concerns, but its atleast worth pointing out that the behavior in pressure spectra examined here may also be present in the Python ATaCR's docs pages (link: https://nfsi-canada.github.io/OBStools/atacr.html#tutorial). There, you see the same behavior in pressure spectra for M08A as described here. M08A's pressure channel is also a DPG.
___

##### PLEASE NOTE: The troubleshooting on this issue is now complete and all analysis in the subsequent cells will utilize the execution of remove_response in `Test-1`.

## `[End of Troubleshooting Addendum]`

        2.3: atacr_transfer_functions | Calculate the tilt and compliance corrective transfer functions.

> Using [atacr_transfer_functions](https://github.com/charleshoots/ATACR_HPS_Comp/blob/512a17e70cc906809bddecbc30767b5a219f0a8a/Packages/ATaCR/OBStools/obstools/scripts/atacr_transfer_functions.py), we build the library of transfer functions used in the tilt and/or compliance correction of the vertical event trace.

> In total, there are six primary transfer functions modeled by ATaCR: ZP, Z1, ZH, ZP-H, Z2-1, and ZP-21. The last three are sets of multiple transfer functions. 

> Before calculating each transfer function, the power and cross spectra are tapered to zero for all frequencies above the station-depth dependent f-notch.

> For the station-averaged event correction used in this example, the ZP-21 transfer is used. 

> ZP-21 is a set of six sub-transfer (denoted by a 'TF_') functions:

1. TF_Z1 = c1Z* / c11
2. TF_21 = c12* / c11
3. TF_P1 = c1P* / c11
4. TF_P2-1 = c2P* - (c12*/c11 x c1P)* / c22 x (1-coh12)
5. TF_Z2-1 = c2Z* - (c12*/c11 x c1Z)* / c22 x (1-coh12)
6. TF_ZP-21 = cZP - (TF_P1 x c1Z)* - (TF_P2-1* x x2Z* - (TF_21 x c12)*) / cPP x (1-coh_1P) x (1-coh_c2cP_c1)

> '*' denotes a complex conjugate, 'c' denotes power/cross spectra, 'coh_' denotes a coherence function.

> Lastly, the tranfser functions are stored and plotted to figures.

>> [Note] Multiple styles of tapering were explored but we ultimately decided to use the same f-notch dependent taper for both compliance and tilt correction transfer functions.  

>> [Note] each are calculated at this step for daily and station-averaged spectra with the exception of ZH and ZP-H which are only made for the daily average correction.

##### [Figure placeholder]
<!-- # Figure: All Transfer Functions (daily and averaged) -->
<!-- ![Figure](znb_images/7D.M07A.transfer_functions_Taper1.png) -->

        2.3: atacr_correct_event | Correct the event for tilt and compliance noise.

> The final step in ATaCR is correcting the event. For this station-averaged correction, we use the ZP-21 transfer function. 

> After calculating the 4-component event spectra using fft (ftZ, ftP, ft1, and ftP),

> The ZP-21 transfer function deconvolves tilt and compliance noise from the vertical event spectra (ftZ) as follow:

Corrected = ftZ - TF_Z1*ft1  -  (ft2-ft1*TF_21)*TF_Z2_1  -  (ftP - ft1*TF_P1 -(ft2 - ft1*TF_21)*TF_P2_1)*TF_ZP_21

> The corrected spectra is returned to the time-domain, plotted, and saved.

##### [Figure placeholder]
# Figure: Corrected event traces plot by ATaCR 
## For the four transfer functions available to a station-averaged spectra.
<!-- ![Figure](znb_images/7D.M07A.2012.181.21.07.sta_corrected.png) -->

### `Phase Three: HPS Correction`

HPS is much simpler to impliment compared to ATaCR.
After loading the 24-hr event SAC file for the vertical trace downloaded and pre-processed in, [run_noisecut](https://github.com/charleshoots/ATACR_HPS_Comp/blob/626fe0a8d825bb781f0cef047c1bab857f01f7a2/Packages/CompCode/ObsQA/TOOLS/io.py#L512) pushes the vertical trace to [Noisecut](https://github.com/charleshoots/ATACR_HPS_Comp/blob/512a17e70cc906809bddecbc30767b5a219f0a8a/Packages/NoiseCut/Source/src/noisecut.py).


> Noisecut has multiple parameters that can be adjusted.

> To remain consistent with Zali et al. (2023), all parameters defined below are hard-set to their published setup.

> Within [Noisecut](https://github.com/charleshoots/ATACR_HPS_Comp/blob/512a17e70cc906809bddecbc30767b5a219f0a8a/Packages/NoiseCut/Source/src/noisecut.py), a power spectrogram is calculated using:

win_length = 163.84 #seconds for the [raw spectrogram](https://github.com/charleshoots/ATACR_HPS_Comp/blob/626fe0a8d825bb781f0cef047c1bab857f01f7a2/Packages/NoiseCut/Source/src/noisecut.py#L131)

hop_length = win_length_sampled // 4  #This corresponds to a 75% overlap.

> All subsequent stages in Noisecut run on partitioned (masked) components of the raw spectrogram.

> Using only the spectrogram from 0.1 to 1hz, a cosine match-filter is then ran to produce an inverted mask of recurrent noise. Combined with the 0.1-1hz partitioned spectrogram yields a spectrogram in this band containing only the observed recurrent noises sources, [S_background](https://github.com/charleshoots/ATACR_HPS_Comp/blob/626fe0a8d825bb781f0cef047c1bab857f01f7a2/Packages/NoiseCut/Source/src/noisecut.py#L175).

> Using only the frequency outside of that same 0.1 to 1hz range, the median filter algorithm (hpss) is then used to produce the spectogram model, [D_harmonic](https://github.com/charleshoots/ATACR_HPS_Comp/blob/626fe0a8d825bb781f0cef047c1bab857f01f7a2/Packages/NoiseCut/Source/src/noisecut.py#L178), containing all observed harmonic noise in this range.

> [S_background](https://github.com/charleshoots/ATACR_HPS_Comp/blob/626fe0a8d825bb781f0cef047c1bab857f01f7a2/Packages/NoiseCut/Source/src/noisecut.py#L175) and [D_harmonic](https://github.com/charleshoots/ATACR_HPS_Comp/blob/626fe0a8d825bb781f0cef047c1bab857f01f7a2/Packages/NoiseCut/Source/src/noisecut.py#L178) are then stacked to yield a full band spectogram of ONLY the observed noise.

> This noise spectogram is then convolved with its original phase and returned to the time-domain, creating a seismic trace of just the noise. 

> Finally, this noise trace is subtracted from the raw time-domain trace to yield a noise corrected output.

### `Phase Four: Compare Results`

> My own library of [helper functions](https://github.com/charleshoots/ATACR_HPS_Comp/blob/main/Packages/CompCode/comp_tools.py) and [classes](https://github.com/charleshoots/ATACR_HPS_Comp/blob/main/Packages/CompCode/ObsQA/OBSM/classes.py) are used to make the subsequent analysis much quicker, scalable, and mutable for comparisons of any ATaCR/HPS output.

 [Comparative_Analysis](https://github.com/charleshoots/ATACR_HPS_Comp/blob/main/Notebooks/Comparative_Analysis.ipynb) has been the primary controller for producing the plots and analysis used in the method comparison.

 First step in the method comparison for a single station-event is simply to plot the corrected outputs side-by-side.

 > Note, when loading the comparisons in for this plot, [Comparative_Analysis](https://github.com/charleshoots/ATACR_HPS_Comp/blob/main/Notebooks/Comparative_Analysis.ipynb) sends a copy of the ATaCR vertical trace output to [run_noisecut](https://github.com/charleshoots/ATACR_HPS_Comp/blob/626fe0a8d825bb781f0cef047c1bab857f01f7a2/Packages/CompCode/ObsQA/TOOLS/io.py#L512) to combine the two methods for this third column of traces.

# Figure: A qualitative comparison of the corrected vertical trace when using either or both of the methods
<!-- ![Figure](TEST_7D.M07A.2012.181.21.07.png) -->

# Figure: Vertical PSD's for each method with the uncorrected trace PSD for reference.
<!-- ![Figure](TEST_7D.M07A.2012.181.21.07_PSD.png) -->

> Next, ZZ, ZP, Z1, and Z2 coherences. 

> ZZ coherence is done between the corrected and uncorrected traces for each method, yielding three coherences in the subplot.

> ZP, Z1, and Z2 coherences are done between the corrected traces and the raw P,1,2 channels. The station averaged ZP, Z1, and Z2 noise coherence is plotted in gray.


.